<h1>Process & Compare Predicted Results to Expected Sequences </h1>

<i>vers. 10/23</i>

In [5]:
import pandas as pd
from tqdm import tqdm

In [6]:
#load all data

#BERT
path_bert = 'path/to/results'
data_bert = pd.read_csv(path_bert, encoding = 'UTF-8')


# BART 
path_bart= 'path/to/results'
path_bart_cd = 'path/to/results'

data_bart = pd.read_csv(path_bart, encoding = 'UTF-8')
data_bart_cd = pd.read_csv(path_bart_cd, encoding = 'UTF-8')


#BELUGA
path_beluga= 'path/to/results'
path_beluga_cd = 'path/to/results'

data_beluga = pd.read_csv(path_beluga, encoding = 'UTF-8')
data_beluga_cd = pd.read_csv(path_beluga_cd, encoding = 'UTF-8')

In [ ]:
#GET REFERENCE LABELS

ref = pd.read_csv('data/daily_dialog_test_next_window3.csv')
print(len(ref))

In [8]:
#LABELS

acts = ['inform', 'question', 'directive', 'commissive']
emos= ['neutral', 'anger', 'disgust', 'fear', 'happiness', 'sadness','surprise']
labels = acts + emos

print(len(labels))

In [ ]:
# GET LABELS FROM REFERENCE + Li (LENGTH OF SEQUENCES)

labels_data = [x.split('+') for x in ref['label']]
li = [len(x) for x in labels_data]

len(labels_data)

<h2>BART</h2>

In [11]:
import re

def find_true_pred(pred, labels):
    match = []
    for x in pred:
        flag = False
        idx = 0
        while idx < len(labels) and not flag:
            label = labels[idx]
            if x in label or label in x or x == label:
                match.append(label)
                flag = True
            
            idx += 1
    
    if not flag:
        print('NO')
        
    return match


def get_labels(data, labels):
    preds = data['hypothese']
    preds = [x.split('+') if '+' in x else x.split(', ') for x in preds]

    clean_preds = []
    c =0

    for pred in preds:
        x = find_true_pred(pred, labels)


        if len(x) <1:
            #print(pred)
            c += 1
        clean_preds.append(x)

    
    print("NUMBR OF EMPTY PREDS : ", c)
                
    return clean_preds

In [ ]:
#GET PREDS BART NO-CD

preds_bart = get_labels(data_bart, labels)

preds_bart[:5]

In [ ]:
#GET PREDS BART CD

preds_bart_cd = get_labels(data_bart_cd, labels)

preds_bart_cd[:10]

In [ ]:
print(len(preds_bart), len(preds_bart_cd))
print(len(labels_data))

<h2>BELUGA</h2>

In [16]:
import re

def get_labels(hypothese, labels):
    flag = False
    idxs = []
    ls = []
    for label in labels:
        if label in hypothese:
            flag = True
            print(hypothese)
            label_idx = [m.start() for m in re.finditer(label, hypothese)]
            for j in label_idx:
                idxs.append(j)
                ls.append(label)

    if flag:    
        sorted_idxs = idxs
        sorted_idxs.sort()
        sorted_labels = [ls[idxs.index(i)] for i in idxs]
    
    else:
        #print('wtf, ', hypothese)
        sorted_labels = ['Empty']

    return sorted_labels


def get_labels_beluga(df, labels):
    empty = 0
    responses = []

    for i, row in tqdm(df.iterrows()):
        hypothese = row['hypothese']
        if isinstance(hypothese, str):
            hypothese = hypothese.strip()
            if len(hypothese) > 3:
                response= get_labels(hypothese, labels)
                responses.append(response)
                if response == ['Empty']: 
                    empty += 1

            else:
                responses.append(['Empty'])
                empty += 1

        else:
            empty += 1
            responses.append(['Empty'])


    print('EMPTY PREDS : ', empty, ' OUT OF ', len(df))
    return responses

In [ ]:
#GET BELUGA NO-CD PREDS

preds_beluga = get_labels_beluga(data_beluga, labels)

In [ ]:
preds_beluga[:20]

In [ ]:
#GET BELUGA CD PREDS

preds_beluga_cd = get_labels_beluga(data_beluga_cd, labels)

<h2>BERT</h2>

In [20]:
def get_scores(df):
    scores = []
    labels = []
    columns = [x for x in df.columns if x not in ['input','reference']]

    for i, row in df.iterrows():
        score = []
        for column in columns:
            score.append(float(row[column]))
        
        sorted_score = sorted(score, reverse=True)
        label = []
        for s in sorted_score:
            idx = score.index(s)
            label.append(columns[idx])
        labels.append(label)
        scores.append([round(x, 3) for x in sorted_score])


    return labels, scores

In [21]:
def thresh_labels(df, threshold = 0.3, k = None, k_list = None):
    preds, scores = get_scores(df)
    new_preds = []
    new_scores = []

    if k_list is not None:
        for i in range(len(preds)):
            new_preds.append(preds[i][:k_list[i]])
            new_scores.append(scores[i][:k_list[i]])

    elif k is not None:
        for i in range(len(preds)):
            new_preds.append(preds[i][:k])
            new_scores.append(scores[i][:k])
    
    else:

        for i in range(len(preds)):
            scores_i = scores[i]
            idx = 0
            flag = True
            while idx < len(scores_i) and flag:
                if scores_i[idx] < threshold:
                    flag = False
                else:
                    idx+= 1
            new_preds.append(preds[i][:idx])
            new_scores.append(scores_i[:idx])
    
    return new_preds, new_scores

In [22]:
#GET BERT NO-CD PREDS
#THRESH = 0.7 FOR BART BASE, 0.5 FOR BART LARGE

preds_bert, scores_bert = thresh_labels(data_bert, threshold = 0.5)

In [ ]:
preds_bert[0]

In [24]:
#GET BERT CD PREDS
#take top li predictions

preds_bert_cd, scores_bert_cd = thresh_labels(data_bert, k_list = li)

In [ ]:
preds_bert_cd[0]

<h2>Random Classifier</h2>

In [ ]:
#Look at distribution of labels sequences length in dataset

dic = {'ones': len([x for x in labels_data if len(x)==1]), 'twos':len([x for x in labels_data if len(x)==2]), 'all': len(labels_data)}
dic

In [28]:
#Randomly select labels according to the distribution

from random import randint, choice
import numpy as np

def random_classifier(data, labels, k = 2, k_list = None):

    labels_random = []        
    for i in range(len(data)):
        k = np.random.choice(np.arange(1, 3), p=[0.8, 0.2])

        turn_labels = []

        #if k_list is not None:
        #    k = k_list[i]

        for j in range(k):
            rand_idx = randint(0, len(labels)-1)
            turn_labels.append(labels[rand_idx])

        labels_random.append(turn_labels)

    return labels_random


preds_random = random_classifier(labels_data, labels, k = 2)
preds_random_li = random_classifier(labels_data, labels, k_list = li)

<h2>Compute Scores</h2>

<h3>Comparison Score</h3>

In [31]:
import statistics

def mean_turn_li(preds):
    lens_labels = []
    c = 0
    nb = 0
    max_l = 0
    min_l = 10000

    for i in range(len(preds)):
        nb +=1
        pred = preds[i]
        lens_labels.append(len(pred))
        if len(pred) ==0:
            c+= 1

        if len(pred) > max_l:
            max_l = len(pred)
            max_i = i

        if len(pred) < min_l:
            min_l = len(pred)
            min_i = i
        
    print("AVERAGE NUMBER OF LABELS PER SPEAKER TURN", sum(lens_labels)/nb)
    print("MAX NUMBER OF LABELS PER SPEAKER TURN", max_l)
    print("MIN NUMBER OF LABELS PER SPEAKER TURN", min_l)
    print("MEDIAN NUMBER OF LABELS PER SPEAKER TURN", statistics.median(lens_labels))

In [ ]:
# GETS STATS

print('REFERENCE')
mean_turn_li(labels_data)

print('')
print('')

print('BERT')
mean_turn_li(preds_bert)

print('')
print('')

print('BERT LI')
mean_turn_li(preds_bert_cd)

print('')
print('')

print('BART')
mean_turn_li(preds_bart)

print('')
print('')

print('BART LI')
mean_turn_li(preds_bart_cd)

print('')
print('')

print('RANDOM')
mean_turn_li(preds_random)


print('')
print('')

print('RANDOM LI')
mean_turn_li(preds_random_li)

print('')
print('')

print('BELUGA')
mean_turn_li(preds_beluga)

print('')
print('')


print('BELUGA LI')
mean_turn_li(preds_beluga_cd)


<h3>Multi-Label F1/Accuracy</h3>

In [33]:
#COMPUTE ONE HOT ENCODE

import numpy as np

def one_hot_encode(preds, labels):
    ohe = np.zeros((len(preds), len(labels)))
    for k in range(len(preds)):
        for i in range(len(preds[k])):
            for j in range(len(labels)):
                if preds[k][i] == labels[j]:
                    ohe[k][j] = 1
                
                else:
                    ohe[k][j] = 0
    return ohe

In [ ]:
# Add Empty for unparsed Beluga predictions

labels += ['Empty']
labels

In [35]:
#CONVERT PREDS TO OHE

labels_bert_ohe = one_hot_encode(preds_bert, labels)
labels_beluga_ohe = one_hot_encode(preds_beluga, labels)
labels_random_ohe = one_hot_encode(preds_random, labels)
labels_bart_ohe = one_hot_encode(preds_bart, labels)


labels_data_ohe = one_hot_encode(labels_data, labels)


labels_bert_li_ohe = one_hot_encode(preds_bert_cd, labels)
labels_beluga_li_ohe = one_hot_encode(preds_beluga_cd, labels)
labels_random_li_ohe = one_hot_encode(preds_random_li, labels)
labels_bart_li_ohe = one_hot_encode(preds_bart_cd, labels)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, classification_report, jaccard_score
from textdistance import levenshtein


print('Jaccard score of beluga : ', jaccard_score(labels_beluga_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_beluga_ohe, labels_data_ohe, target_names= labels))


levsim = levenshtein.normalized_similarity(preds_beluga,labels_data)

print('LEVENSHTEIN SIM beluga : ', levsim)

In [ ]:
print('Jaccard score of beluga li: ', jaccard_score(labels_beluga_li_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_beluga_li_ohe, labels_data_ohe, target_names= labels))


levsim = levenshtein.normalized_similarity(preds_beluga_cd,labels_data)

print('LEVENSHTEIN SIM beluga li: ', levsim)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, classification_report, jaccard_score
from textdistance import levenshtein


print('Jaccard score of multilabel bert : ', jaccard_score(labels_bert_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_bert_ohe, labels_data_ohe, target_names= labels))


levsim = levenshtein.normalized_similarity(preds_bert,labels_data)

print('LEVENSHTEIN SIM BERT : ', levsim)

In [ ]:
print('Jaccard score of multilabel bert li: ', jaccard_score(labels_bert_li_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_bert_li_ohe, labels_data_ohe, target_names= labels))


levsim = levenshtein.normalized_similarity(preds_bert_cd,labels_data)

print('LEVENSHTEIN SIM BERT LI: ', levsim)

In [ ]:
print('Jaccard score of bart ohe: ', jaccard_score(labels_bart_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_bart_ohe, labels_data_ohe, target_names= labels))

levsim = levenshtein.normalized_similarity(preds_bart,labels_data)

print('LEVENSHTEIN SIM BART : ', levsim)

In [ ]:
print('Jaccard score of bart  ohe li: ', jaccard_score(labels_bart_li_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_bart_li_ohe, labels_data_ohe, target_names= labels))

levsim = levenshtein.normalized_similarity(preds_bart_cd,labels_data)

print('LEVENSHTEIN SIM BART LI: ', levsim)

In [ ]:
print('Jaccard score of random : ', jaccard_score(labels_random_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_random_ohe, labels_data_ohe, target_names= labels))

levsim = levenshtein.normalized_similarity(preds_random,labels_data)

print('LEVENSHTEIN SIM RANDOM : ', levsim)

In [ ]:
print('Jaccard score of random li: ', jaccard_score(labels_random_li_ohe, labels_data_ohe, average = 'weighted'))

print(classification_report(labels_random_li_ohe, labels_data_ohe, target_names= labels))

levsim = levenshtein.normalized_similarity(preds_random_li,labels_data)

print('LEVENSHTEIN SIM RANDOM LI: ', levsim)